In [1]:
import medmnist
from medmnist import INFO
from medmnist.dataset import DermaMNIST

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import dataset_without_pytorch
from dataset_without_pytorch import get_loader

from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
import cnn

In [2]:
data_flag = 'dermamnist'
download = True

BATCH_SIZE = 128 #Lote tamanio

info_28 = INFO[data_flag] # Info

# Obtener dinamicamente la clase del conjunto de datos desde un modulo
DataClass_28 = getattr(dataset_without_pytorch, info_28['python_class'])

train_dataset_28 = DataClass_28(split='train', download=download)

# Encapsular los datos en un DataLoader para hacer lotes
train_loader_28 = get_loader(dataset=train_dataset_28, batch_size=BATCH_SIZE)

Using downloaded and verified file: C:\Users\abiab\.medmnist\dermamnist.npz


In [3]:
images = train_dataset_28.imgs
labels = train_dataset_28.labels

images_class_5 = [image for image, label in zip(images, labels) if label == 5]

In [4]:
flattened_images = [image.flatten() for image in images_class_5]

# Aplicar K-means con 1000 clusters
kmeans = KMeans(n_clusters=1000, random_state=42)
kmeans.fit(flattened_images)

# Obtener los centros de los clusters
cluster_centers = kmeans.cluster_centers_

# Para cada centroide, encontrar la imagen más cercana en el conjunto original
selected_images = []
for center in cluster_centers:
    # Calcular la distancia entre el centroide y todas las imágenes
    distances = np.linalg.norm(flattened_images - center, axis=1)
    # Encontrar el índice de la imagen más cercana
    closest_image_index = np.argmin(distances)
    # Añadir la imagen más cercana a la lista de imágenes seleccionadas
    selected_images.append(images_class_5[closest_image_index])

c:\Users\abiab\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\abiab\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [5]:
# Filtrar imágenes y etiquetas de todas las clases excepto la clase 5
images_other_classes = [image for image, label in zip(images, labels) if label != 5]
labels_other_classes = [label for label in labels if label != 5]

# Crear etiquetas para las imágenes seleccionadas de la clase 5
selected_labels = [5] * len(selected_images)

# Combinar las imágenes y etiquetas de las otras clases con las seleccionadas de la clase 5
new_images = images_other_classes + selected_images

selected_labels = [[x] for x in selected_labels]

new_labels = labels_other_classes + selected_labels

In [6]:
new_labels = [int(label[0]) if isinstance(label, (list, np.ndarray)) else int(label) for label in new_labels]

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator


new_images = np.array(new_images)
new_labels = np.array(new_labels)

# Calcular la distribución de clases
unique, counts = np.unique(new_labels, return_counts=True)
class_distribution = dict(zip(unique, counts))

# Calcular el promedio de imágenes por clase
average_images = int(np.mean(list(class_distribution.values())))

# Crear un generador de datos para rotación
datagen = ImageDataGenerator(rotation_range=40)

# Listas para las imágenes y etiquetas aumentadas
augmented_images = []
augmented_labels = []

# Aumentar imágenes en las clases 3 y 6 hasta alcanzar el promedio
target_classes = [3, 6]
for target_class in target_classes:
    class_images = new_images[new_labels == target_class]
    current_count = class_distribution[target_class]
    images_to_add = average_images - current_count

    if images_to_add > 0:
        for i in range(images_to_add):
            # Seleccionar una imagen aleatoria de la clase
            img = class_images[i % len(class_images)]
            img = np.expand_dims(img, 0)  # Expande dimensiones para el generador

            # Generar una imagen aumentada
            for batch in datagen.flow(img, batch_size=1):
                augmented_images.append(batch[0].astype(np.uint8))  # Agregar imagen aumentada
                augmented_labels.append(target_class)
                break  # Solo necesitamos una imagen por iteración

# Agregar las imágenes aumentadas al conjunto original
new_images = np.concatenate([new_images, np.array(augmented_images)])
new_labels = np.concatenate([new_labels, np.array(augmented_labels)])

In [7]:

# Red1 = cnn.Propuesta_1(new_images, new_labels)
# Red1.CrearModelo()
# Red1.EvaluarModelo()

In [8]:
# Red2 = cnn.Propuesta_2(new_images, new_labels)
# Red2.CrearModelo()
# Red2.EvaluarModelo()

In [9]:
# Red3 = cnn.Propuesta_3(new_images, new_labels)
# Red3.CrearModelo()
# Red3.EvaluarModelo()

In [10]:
# Red4 = cnn.Propuesta_4(new_images, new_labels)
# Red4.CrearModelo()
# Red4.EvaluarModelo()

In [11]:
# Red5 = cnn.Propuesta_5 (new_images, new_labels)
# Red5.CrearModelo()
# Red5.EvaluarModelo()

In [12]:
RedConv = cnn.RedNeuronal(new_images, new_labels)
RedConv.EvaluarModelos()



c:\Users\abiab\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.1887 - loss: 2.8248 - val_accuracy: 0.1621 - val_loss: 2.2678
Epoch 2/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.2767 - loss: 2.2083 - val_accuracy: 0.1621 - val_loss: 2.3913
Epoch 3/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3165 - loss: 2.0347 - val_accuracy: 0.2508 - val_loss: 2.1122
Epoch 4/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3239 - loss: 1.9936 - val_accuracy: 0.3547 - val_loss: 1.8813
Epoch 5/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3586 - loss: 1.9289 - val_accuracy: 0.3639 - val_loss: 1.8181
Epoch 6/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.3513 - loss: 1.9239 - val_accuracy: 0.3823 - val_loss: 1.7665
Epoch 7/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3888 - loss: 1.8207 - val_accuracy: 0.4251 - val_loss: 1.7385
Epoch 8/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3822 - loss: 1.8216 - 

In [ ]:
test_dataset_28 = DataClass_28(split='test', download=download)
test_loader_28 = get_loader(dataset=test_dataset_28, batch_size=BATCH_SIZE)

paraPred_images = np.array(test_dataset_28.imgs)
paraPred_labels = np.array(test_dataset_28.labels)

Using downloaded and verified file: C:\Users\abiab\.medmnist\dermamnist.npz


In [ ]:
valoresPredichos = RedConv.Predecir(paraPred_images)

precision = accuracy_score(paraPred_labels, valoresPredichos) * 100
print(f"Precisión: {precision:.2f}%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Precisión: 2.00%
